In [269]:
import pandas as pd

import numpy as np
from scipy.stats import norm
import queue
from ctypes import string_at
from sys import getsizeof
import ctypes
from binascii import hexlify
import operator
%matplotlib inline
#以上操作导入包

In [270]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn import tree

In [271]:
df_train = pd.read_csv(r"C:\Users\fulian\Desktop\tarinset.csv",encoding='utf-8')
df_test = pd.read_csv(r"C:\Users\fulian\Desktop\test.csv",encoding='utf-8')
df_train = df_train.drop(df_train.columns[[0]], axis=1)
df_test = df_test.drop(df_test.columns[[0]], axis=1)

In [272]:
def featrue_split_list(df):
    c = []
    for i in range(0,df.shape[1] -1):
         featrues = set()
         for j in range(0,df.shape[0]):
            featrues.add(df.iat[j,i])
         a = list(featrues)
         
         c.append(a)
    
    return c
featrues = featrue_split_list(df_train)
l = (len(featrues))
print(featrues)

[[' 浅白', ' 乌黑', ' 青绿'], [' 硬挺', ' 稍蜷', ' 蜷缩'], [' 清脆', ' 沉闷', ' 浊响'], [' 清晰', ' 模糊', ' 稍糊'], [' 平坦', ' 稍凹', ' 凹陷'], [' 硬滑', ' 软粘']]


In [273]:
def B(x):
    return -x*np.log2(x)-(1-x)*np.log2(1-x)

In [274]:
class tree:
    def __init__(self,val):
        self.key = val
        self.left = None
        self.right = None
        self.middle = None
        self.root = val
    def insert(self,val):
      if isinstance(val,tree):
          if self.key != None:
             if self.left is None:
               self.left = val
            
             elif self.middle is None:
               self.middle = val
            
             elif self.right is None:
                self.right = val
          else:
            self.key = val
      else:
       if self.key != None:
         if self.left is None:
            self.left = tree(val)
            
         elif self.middle is None:
            self.middle = tree(val)
            
         elif self.right is None:
            self.right = tree(val)
              
         
       else:
         self.key = val
    



    def printValues(self):
       if self.left:
         self.left.printValues()
       print(self.key)
       if self.middle:
          self.middle.printValues()
       if self.right:
          self.right.printValues()
    def printlayer(self):
       q=queue.Queue(100)
       q.put(self)
       while q.empty() ==False:
          p = q.get()
          print(p.key)
          if p.left != None:
             q.put(p.left)
          if p.middle != None:
             q.put(p.middle)
          if p.right != None:
             q.put(p.right)
          
          

    


    



In [275]:
def all_split(df,a):
    i = 0
    for x in range(df.shape[0]):
        for j in range(df.shape[1]):
            if df.iat[x,j] == a:
                i = j
    data_split =  pd.DataFrame()
    for u in range(0,df.shape[0]):
         if df.iat[u,i] == a:
            data_split =data_split.append(df.iloc[u,:])
    return data_split   


     


In [276]:

listexp = [' 是',' 否']
def Ginichoose(parent,a):
    min = 1000
    best = 0
    for i in range(len(a)):
        sum2 = 0
        Gini = 0
        split = pd.DataFrame()
        for j in range(len(a[i])):
            sumsub = 0
            split = all_split(parent,a[i][j])
            for u in range(split.shape[0]):
                if split.iat[u,split.shape[1] - 1] == listexp[0]:
                    sumsub += 1
            if split.shape[0] != 0:
                sum2 = 1- (sumsub/split.shape[0])*(sumsub/split.shape[0])-((split.shape[0]-sumsub)/split.shape[0])*((split.shape[0]-sumsub)/split.shape[0])
            Gini = Gini + sum2*(split.shape[0]/parent.shape[0])
        
        if Gini < min :
            best = i
            min = Gini
    
    return a[best]


In [277]:
def delet(featrues_pre,featrues_deleted):
    f = -1
    for i in range(len(featrues_pre)):
        for j in range(len(featrues_pre[i])):
            if featrues_pre[i][j] == featrues_deleted[0]:
                f =i
                break
    if f>=0:
        del featrues_pre[f]
      
    return featrues_pre
 


In [278]:
def same_class(df):
    for i in range(df.shape[0]):
        if df.iat[i,df.shape[1] - 1] != df.iat[0,df.shape[1] - 1]:
            return False
    return True

    

In [279]:
def mode(df):
    listresult =[' 是',' 否']
    sum1 = 0
    sum2 = 0
    list_mode = []
    for i in range(df.shape[0]):
        if df.iat[i,df.shape[1] - 1] == listresult[0]:
            
            sum1 += 1
        else:
            sum2 += 1
    if sum1 > sum2:
        list_mode.append(listresult[0])
    else:
        list_mode.append(listresult[1])  
    list_mode_result = pd.DataFrame(list_mode)
    return list_mode




In [280]:
def DTL(examples,featrues,parent_examples):
    if examples.empty == True:
        return parent_examples
    elif same_class(examples) == True:
        return mode(examples)
    elif len(featrues) == 0:
        return mode(examples)
    else:
        
        best = Ginichoose(examples,featrues)
        Tree = tree(best)
        root = Tree
        for q in range(len(best)):
            examplesub = all_split(examples,best[q])
            temp = []
            temp.append(best[q])
            subtree = DTL(examplesub,delet(featrues,best),mode(examples))
            
            if Tree.left == None:
                Tree.insert(temp)
                Tree = Tree.left

            elif Tree.middle == None:
                Tree.insert(temp)
                Tree = Tree.middle

            elif Tree.right == None:
                Tree.insert(temp)
                Tree = Tree.right
            Tree.insert(subtree)
            Tree = root  
            
        return Tree



In [281]:

print("未剪枝决策树")
DTL(df_train,featrue_split_list(df_train),None).printlayer()
# Full_tree = DTL(df_train,featrue_split_list(df_train),None)
# Full_tree.printlayer()


未剪枝决策树
[' 浅白', ' 乌黑', ' 青绿']
[' 浅白']
[' 乌黑']
[' 青绿']
[' 否']
[' 硬挺', ' 稍蜷', ' 蜷缩']
[' 清脆', ' 沉闷', ' 浊响']
[' 硬挺']
[' 稍蜷']
[' 蜷缩']
[' 清脆']
[' 沉闷']
[' 浊响']
[' 是']
[' 清晰', ' 模糊', ' 稍糊']
[' 是']
[' 否']
[' 否']
[' 是']
[' 清晰']
[' 模糊']
[' 稍糊']
[' 否']
[' 否']
[' 是']


In [282]:
global p 
p = []

In [283]:
def all_split_index(df,a):
    index = []
    for x in range(df.shape[0]):
        for j in range(df.shape[1]):
            if df.iat[x,j] == a:
                index.append(x)
    
    return index 


In [284]:
def if_pruning(df):
   
    if len(p) == 1:
        s = 0
        for i in range(len(p[0])):
            t1 = all_split(df,p[0][i])
            for j in range(t1.shape[0]):
                if t1.iat[j,t1.shape[1] - 1] == mode(t1)[0]:
                    s += 1
        ratio1 = s/df_train.shape[0]
    else:
        a = p[1]
        b = p[0]
        parent_split = all_split_index(df,a)
        s = 0
        for j in range(len(b)):
            child_split = all_split_index(df,b[j])
            t1 = all_split(df,a)
            t = all_split(t1,b[j])
            
            for i in range(df.shape[0]):
                if i in parent_split and i in child_split:
                    if df.iat[i,df.shape[1] - 1] == mode(t)[0]:
                        s += 1
            for i in range(df.shape[0]):
                if i in parent_split and i not in child_split:
                    if df.iat[i,df.shape[1] - 1] == mode(t)[0]:
                        s += 1
        ratio1 = s/df_train.shape[0]
    
    return ratio1

    

In [285]:

def DTLpre(examples,featrues,parent_examples,ratio0):
    if examples.empty == True:
        return parent_examples
    elif same_class(examples) == True:
        return mode(examples)
    elif len(featrues) == 0:
        return mode(examples)
    else:
        best = Ginichoose(examples,featrues)
        p.append(best)
        
        ratio = if_pruning(df_train)
        if ratio < ratio0:
            return mode(examples)
        p.clear()
        Tree = tree(best)
        root = Tree
        for q in range(len(best)):
            examplesub = all_split(examples,best[q])
            p.append(best[q])
            subtree = DTLpre(examplesub,delet(featrues,best),mode(examples),ratio)
            
            if Tree.left == None:
                Tree.insert(best[q])
                Tree = Tree.left

            elif Tree.middle == None:
                Tree.insert(best[q])
                Tree = Tree.middle

            elif Tree.right == None:
                Tree.insert(best[q])
                Tree = Tree.right
            
            Tree.insert(subtree)
            #p.clear()
            Tree = root  
        return Tree



In [286]:
print("预剪枝决策树")
DTLpre(df_train,featrue_split_list(df_train),None,3/7).printlayer()



预剪枝决策树
[' 浅白', ' 乌黑', ' 青绿']
 浅白
 乌黑
 青绿
[' 否']
[' 是']
[' 否']


In [287]:

def binaryTreePaths(root):
    """
    :type root: TreeNode
    :rtype: List[str]
    """
    def construct_paths(root, path):
        if root:
            path += str(root.key)
            if not root.left and not root.right:  # 当前节点是叶子节点
                paths.append(path)  # 把路径加入到答案中
            else:
                path += '->'  # 当前节点不是叶子节点，继续递归遍历
                construct_paths(root.left, path)
                construct_paths(root.middle, path)
                construct_paths(root.right, path)

    paths = []
    construct_paths(root, '')
    return paths

In [288]:
def change_single(str):
    s = ''
    y = []
    for i in range(len(str)):
        if str[i] != '\'' and str[i] != '[' and str[i] != ']':
            s = s + str[i]
    y.append(s)
    return y

In [289]:
def change_list(paths):
    last = []
    for u in range(len(paths)):
        result = []
        t = paths[u].split('->')
        for j in range(len(t)):
            if len(t[j]) == 7:
                x = change_single(t[j])
                result.append(x)
        last.append(result)
    return last

In [290]:
def return_sameclass(a):
    x = featrue_split_list(df_train)
    for i in range(len(x)):
        if a[0] in x[i]:
            return x[i]

In [291]:
Full_tree = DTL(df_train,featrue_split_list(df_train),None)
T =binaryTreePaths(Full_tree)
# print(T)
change_list(T)


[[[' 浅白']],
 [[' 乌黑'], [' 硬挺']],
 [[' 乌黑'], [' 稍蜷'], [' 清晰']],
 [[' 乌黑'], [' 稍蜷'], [' 模糊']],
 [[' 乌黑'], [' 稍蜷'], [' 稍糊']],
 [[' 乌黑'], [' 蜷缩']],
 [[' 青绿'], [' 清脆']],
 [[' 青绿'], [' 沉闷']],
 [[' 青绿'], [' 浊响']]]

In [292]:
w = []
def post_pruning(results):
    for i in range(len(results)):
        while len(results[i]) > 1:
            l = len(results[i])
            flag = 0
            train = df_train
            test = df_test
            sum = 0
            for j in range(l - 1):
                train = all_split(train,results[i][j][0])
                test = all_split(test,results[i][j][0])
            temp = return_sameclass(results[i][l - 1])
            prun_train = train
            prun_test = test
            if test.empty == True:
                flag = 1
            for u in range(len(temp)):
                train = all_split(prun_train,temp[u])
                test = all_split(prun_test,temp[u])
                for v in range(test.shape[0]):
                    if test.iat[v,test.shape[1] - 1] != mode(train)[0]:
                        sum += 1
            prun = 0
            for u in range(prun_test.shape[0]):
                if prun_test.iat[u,prun_test.shape[1] - 1] != mode(prun_train)[0]:
                    prun += 1
            if prun >= sum:
                break
            elif prun < sum and flag == 0:
                results[i].pop()
    return results




In [293]:
w1 = post_pruning(change_list(T))
# print(w1)
def removesame(lists):
    i = 0
    while True:
        if i == len(lists): break
        if lists.count(lists[i]) != 1:
            lists.pop(i)
            continue
        else:
            i += 1
    
    for i in range(len(lists)):
        l = len(lists[i])
        train = df_train
        for j in range(l):
           train = all_split(train,lists[i][j][0])
        lists[i].append(mode(train))
    w = []
    for i in range(len(lists)):
        t = []
        l = len(lists[i])
        for j in range(l):
            if lists[i][j][0] != listexp[0] and lists[i][j][0] != listexp[1]:
                t.append(return_sameclass(lists[i][j]))
            t.append(lists[i][j])
            
        w.append(t)
    return w

#removesame(w1)

In [294]:
def growtree(lists,Tree):
    root = Tree
    for i in range(len(lists)):
        Tree = root
        for j in range(1,len(lists[i])):
            if Tree.left != None and Tree.left.key == lists[i][j]:
                Tree = Tree.left
            elif Tree.middle != None and Tree.middle.key == lists[i][j]:
                Tree = Tree.middle
            elif Tree.right != None and Tree.right.key == lists[i][j]:
                Tree = Tree.right
            else:
                Tree.left.key = lists[i][j]
                Tree = Tree.left
                Tree.right = None
                Tree.middle = None
                Tree.left = None
    Tree = root
    return Tree




In [295]:
print("后剪枝决策树")
growtree(removesame(w1),Full_tree).printlayer()

[[[' 浅白', ' 乌黑', ' 青绿'], [' 浅白'], [' 否']], [[' 浅白', ' 乌黑', ' 青绿'], [' 乌黑'], [' 硬挺', ' 稍蜷', ' 蜷缩'], [' 硬挺'], [' 否']], [[' 浅白', ' 乌黑', ' 青绿'], [' 乌黑'], [' 硬挺', ' 稍蜷', ' 蜷缩'], [' 稍蜷'], [' 否']], [[' 浅白', ' 乌黑', ' 青绿'], [' 乌黑'], [' 硬挺', ' 稍蜷', ' 蜷缩'], [' 蜷缩'], [' 是']], [[' 浅白', ' 乌黑', ' 青绿'], [' 青绿'], [' 否']]]
[' 浅白', ' 乌黑', ' 青绿']
[' 浅白']
[' 乌黑']
[' 青绿']
[' 否']
[' 硬挺', ' 稍蜷', ' 蜷缩']
[' 否']
[' 硬挺']
[' 稍蜷']
[' 蜷缩']
[' 否']
[' 否']
[' 是']
